In [1]:
import pickle
import re

import pandas as pd

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
examples = {}

for i in range(0, 5):
    with open("../../out/definitions.pickle", "rb") as definitions_file:
        with open(f"../../out/llama/examples_{i}_shot.pickle", "rb") as examples_file:
            examples[i] = {}
            cur_definitions = pickle.load(definitions_file)
            cur_examples = pickle.load(examples_file)
            for (d, de) in cur_definitions.items():
                # if i == 0:
                #     examples[i][d] = (de, [], ex)
                # else:
                examples[i][d] = (de, *cur_examples[d])
    examples[i] = pd.DataFrame.from_dict(examples[i], orient="index", columns=["definition", "examples", "generated"])
    
examples[0]

,definition,examples,generated
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e..."
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die beiden Pferde sind gleicher Größe."", ""Di..."
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Michael is the tallest"", ""John has the short..."
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""The athlete is the fastest runner on the tea..."
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt wie eine Lokomotive"", ""Das Ha..."
...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der höher ist als der Frau"", ""Ein ..."
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""Xoder"", ""Y""]"
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo das Licht ist, ist auch die Dunkelheit"", ..."
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""Die 'äquativ' ist gleich 22"", ""Das Bild ist ..."


## Llama-2:
### Anzahl Beispielsätze:

In [3]:
def evaluate_string(s):
    try:
        evaluated_value = eval(s)
        
        if isinstance(evaluated_value, list):
            
            return [str(e) for e in evaluated_value]  # muss String sein!
        
    except:# (SyntaxError, ValueError):
        return [s.replace("[", "").replace("]", "")]

In [4]:
for example_group in examples.values():
    example_group['evaluated_generated'] = example_group['generated'].apply(evaluate_string)
    example_group['num_generated'] = example_group['evaluated_generated'].apply(len)

examples[0]

,definition,examples,generated,evaluated_generated,num_generated
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e...",[Die meisten von ihnen haben zuvor noch nie ei...,2
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die beiden Pferde sind gleicher Größe."", ""Di...","[Die beiden Pferde sind gleicher Größe., Die b...",2
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Michael is the tallest"", ""John has the short...","[Michael is the tallest, John has the shortest...",3
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""The athlete is the fastest runner on the tea...",[The athlete is the fastest runner on the team.],1
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt wie eine Lokomotive"", ""Das Ha...","[Das Auto fährt wie eine Lokomotive, Das Haus ...",3
...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der höher ist als der Frau"", ""Ein ...","[Ein Mann, der höher ist als der Frau, Ein Man...",3
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""Xoder"", ""Y""]","[Xoder, Y]",2
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo das Licht ist, ist auch die Dunkelheit"", ...","[Wo das Licht ist, ist auch die Dunkelheit, Wo...",4
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""Die 'äquativ' ist gleich 22"", ""Das Bild ist ...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",2


In [5]:
examples[0]["num_generated"].mean()

2.829383886255924

In [6]:
examples[0]["num_generated"].mode()  # Häufigste Anzahl Beispielsätze (Modalwert)

0    3
Name: num_generated, dtype: int64

### Anzahl unterschiedlicher Beispielsätze

In [7]:
for example_group in examples.values():
    example_group['unique_generated'] = example_group['evaluated_generated'].apply(lambda x: (len(set(x))/len(x)) if len(x) > 0 else 0.0)

In [8]:
examples[0]

,definition,examples,generated,evaluated_generated,num_generated,unique_generated
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e...",[Die meisten von ihnen haben zuvor noch nie ei...,2,1.0
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die beiden Pferde sind gleicher Größe."", ""Di...","[Die beiden Pferde sind gleicher Größe., Die b...",2,1.0
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Michael is the tallest"", ""John has the short...","[Michael is the tallest, John has the shortest...",3,1.0
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""The athlete is the fastest runner on the tea...",[The athlete is the fastest runner on the team.],1,1.0
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt wie eine Lokomotive"", ""Das Ha...","[Das Auto fährt wie eine Lokomotive, Das Haus ...",3,1.0
...,...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der höher ist als der Frau"", ""Ein ...","[Ein Mann, der höher ist als der Frau, Ein Man...",3,1.0
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""Xoder"", ""Y""]","[Xoder, Y]",2,1.0
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo das Licht ist, ist auch die Dunkelheit"", ...","[Wo das Licht ist, ist auch die Dunkelheit, Wo...",4,1.0
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""Die 'äquativ' ist gleich 22"", ""Das Bild ist ...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",2,1.0


In diesen Ausgaben unterscheiden sich die Anzahl der generierten Beispiele und die Anzahl der tatsächlich unterschiedlichen Beispiele:

In [9]:
not_all_unique = examples[0][(0.0 < examples[0]['unique_generated']) & (examples[0]['unique_generated'] < 1.0)]
print(len(not_all_unique) / len(examples[0]) * 100, "%")
not_all_unique

6.6350710900473935 %


,definition,examples,generated,evaluated_generated,num_generated,unique_generated
1029,Bei der in der Forschung als „w-Schlagzeile“ (...,[],"[""Mehr Artikel anzeigen"", ""Kaufen"", ""Buch kauf...","[Mehr Artikel anzeigen, Kaufen, Buch kaufen, B...",8,0.375000
1054,"Die ""Intensivierung:ART_HERRSCHERTITEL_ART_NP-...",[],"[""Königin"", ""Kaiser"", ""Papst"", ""Königin"", ""Kai...","[Königin, Kaiser, Papst, Königin, Kaiserin]",5,0.800000
1134,"Die ""Additiv_Koordination:sowohlXalsauchY"" umf...",[],"[""Dieses Auto fährt sowohl schnell als auch si...",[Dieses Auto fährt sowohl schnell als auch sic...,4,0.250000
12,"Die ""Negation:wederXPnochXP-Konstruktion"" umfa...",[],"[""Es wird gesagt, dass jeder in den Bergen wei...","[Es wird gesagt, dass jeder in den Bergen weil...",2,0.500000
129,"Die ""Äquativ:eine_ArtX-Konstruktion"" gehört zu...",[],"[""Eine Art von Reis ist nach dem Kochen äquiva...",[Eine Art von Reis ist nach dem Kochen äquival...,9,0.444444
1324,"Die Konstruktion"" Intensivierung_Attribut_nach...",[],"[""intensivierte"", ""satt"", ""satt""]","[intensivierte, satt, satt]",3,0.666667
1346,Die meist als Prädikativum verwendete Konstruk...,[],"[""Prototyp-Buchesteht"", ""Prototyp-Buchesteht""]","[Prototyp-Buchesteht, Prototyp-Buchesteht]",2,0.500000
1351,"Die ""Reduplikation_Irrelevanz:N1hinN1her-Konst...",[],"[""Dieser Konstrukt ist identisch mit diesem Ko...",[Dieser Konstrukt ist identisch mit diesem Kon...,2,0.500000
1777,"Die ""Korrelativer_Äquativ:VPin_dem_MaßeXP-Kons...",[],"[""In dem Maß des Gleichen"", ""Im selben Maß"", ""...","[In dem Maß des Gleichen, Im selben Maß, In de...",9,0.666667
1842,"Die ""Personenreferenz_abwertend:x-ling-Konstru...",[],"[""Abwertungs-lingua"", ""Abwertungs-Lingua"", ""Ab...","[Abwertungs-lingua, Abwertungs-Lingua, Abwertu...",6,0.500000


### Tatsächlich neue Inhalte:

In [10]:
stopwords = {s.lower() for s in stopwords.words("german")}  # Stoppwörter werden entfernt, weil die nichts über Wiederholungen aussagen

def calculate_overlap(row):
    generated_set = {w for w in " ".join(row['evaluated_generated']).split() if w.lower() not in stopwords}
    
    if len(generated_set) > 0:
        definition_overlap = len(generated_set.intersection({w for w in row['definition'].split() if w.lower() not in stopwords})) / len(generated_set)
        examples_overlap = len(generated_set.intersection({w for w in " ".join(row['examples']).split() if w.lower() not in stopwords})) / len(generated_set)
        overlap = len(generated_set.intersection({w for w in row['definition'].split() if w.lower() not in stopwords} | {w for w in " ".join(row['examples']).split() if w.lower() not in stopwords})) / len(generated_set)
    else:
        definition_overlap = 0.0
        examples_overlap = 0.0
        overlap = 0.0
    
    return definition_overlap, examples_overlap, overlap

In [11]:
for example_group in examples.values():
    example_group[['definition_overlap', 'examples_overlap', 'overlap']] = example_group.apply(calculate_overlap, axis=1, result_type='expand')

In [12]:
examples[0]

,definition,examples,generated,evaluated_generated,num_generated,unique_generated,definition_overlap,examples_overlap,overlap
10,"Die ""Negation:NEG_XgeschweigedennY-Konstruktio...",[],"[""Die meisten von ihnen haben zuvor noch nie e...",[Die meisten von ihnen haben zuvor noch nie ei...,2,1.0,1.000000,0.0,1.000000
100,"Die ""Äquativ_Plural-Konstruktion"" gehört zu de...",[],"[""Die beiden Pferde sind gleicher Größe."", ""Di...","[Die beiden Pferde sind gleicher Größe., Die b...",2,1.0,0.166667,0.0,0.166667
1004,"Die ""Superlativ_Klimax:ADJ1_ADJ1-er_NP-Konstru...",[],"[""Michael is the tallest"", ""John has the short...","[Michael is the tallest, John has the shortest...",3,1.0,0.000000,0.0,0.000000
1006,"Die ""Superlativ:PRÄP_ADJ-ster_NP-Konstruktion""...",[],"[""The athlete is the fastest runner on the tea...",[The athlete is the fastest runner on the team.],1,1.0,0.000000,0.0,0.000000
101,"Die ""Äquativ:ADJwieNP-Konstruktion"" gehört zu ...",[],"[""Das Auto fährt wie eine Lokomotive"", ""Das Ha...","[Das Auto fährt wie eine Lokomotive, Das Haus ...",3,1.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
97,"Die ""Komparativ:ADJ1-eralsADJ1-Konstruktion"" g...",[],"[""Ein Mann, der höher ist als der Frau"", ""Ein ...","[Ein Mann, der höher ist als der Frau, Ein Man...",3,1.0,0.142857,0.0,0.142857
973,"Bei ""Disjunktion_Doppeltitel:XoderY"" handelt e...",[],"[""Xoder"", ""Y""]","[Xoder, Y]",2,1.0,0.000000,0.0,0.000000
976,"Bei ""Korrelation_Affirmation:WoXist, istY"" han...",[],"[""Wo das Licht ist, ist auch die Dunkelheit"", ...","[Wo das Licht ist, ist auch die Dunkelheit, Wo...",4,1.0,0.222222,0.0,0.222222
98,"Die ""Äquativ:soADJwieXP-Konstruktion"" gehört z...",[],"[""Die 'äquativ' ist gleich 22"", ""Das Bild ist ...","[Die 'äquativ' ist gleich 22, Das Bild ist äqu...",2,1.0,0.000000,0.0,0.000000


In [13]:
examples[0]["overlap"].mean()

0.04698239684300459

In [14]:
for i, example_group in examples.items():
    example_group.to_csv(f"../../out/llama/{i}_shot_data.tsv", sep="\t", decimal=",")
    example_group.to_excel(f"../../out/llama/{i}_shot_data.xlsx")